In [ ]:
"""
Grover's Algorithm in Qiskit - Pseudocode Structure

This shows how to implement Grover's algorithm in actual Qiskit
when the library is available.
"""

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
import numpy as np
from qiskit import transpile
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def create_oracle(n_qubits, marked_items):
    oracle = QuantumCircuit(n_qubits, name="Oracle")
    for item in marked_items:
        binary = format(item, f'0{n_qubits}b')
        flip_qubits = [i for i, b in enumerate(binary) if b == '0']
        for q in flip_qubits:
            oracle.x(q)
        # MCX pattern for multi-controlled-Z
        oracle.h(n_qubits-1)
        controls = list(range(n_qubits-1))
        target = n_qubits-1
        if controls:
            oracle.mcx(controls, target)
        else:
            oracle.z(target)
        oracle.h(n_qubits-1)
        for q in flip_qubits:
            oracle.x(q)
    return oracle

def create_diffusion_operator(n_qubits):
    diffusion = QuantumCircuit(n_qubits, name="Diffusion")
    diffusion.h(range(n_qubits))
    diffusion.x(range(n_qubits))
    diffusion.h(n_qubits-1)
    controls = list(range(n_qubits-1))
    target = n_qubits-1
    if controls:
        diffusion.mcx(controls, target)
    else:
        diffusion.z(target)
    diffusion.h(n_qubits-1)
    diffusion.x(range(n_qubits))
    diffusion.h(range(n_qubits))
    return diffusion


def grovers_algorithm_qiskit(n_qubits, marked_items, shots=1024):
    N = 2**n_qubits
    M = len(marked_items)
    optimal_iterations = int(np.round(np.pi / 4 * np.sqrt(N / M)))
    print(f"Database size N = {N}")
    print(f"Number of qubits = {n_qubits}")
    print(f"Marked items: {marked_items}")
    print(f"Optimal Grover iterations: {optimal_iterations}\n")

    qreg = QuantumRegister(n_qubits, 'q')
    creg = ClassicalRegister(n_qubits, 'c')
    circuit = QuantumCircuit(qreg, creg)

    # Step 1: Uniform superposition
    circuit.h(qreg)
    circuit.barrier()

    # Prepare gates
    oracle_gate = create_oracle(n_qubits, marked_items).to_gate(label="Oracle")
    diffusion_gate = create_diffusion_operator(n_qubits).to_gate(label="Diffusion")

    # Step 2: Grover iterations
    for i in range(optimal_iterations):
        print(f"Iteration {i+1}: Applying Oracle and Diffusion...")
        circuit.append(oracle_gate, qreg)
        circuit.barrier()
        circuit.append(diffusion_gate, qreg)
        circuit.barrier()

    # Step 3: Measure
    circuit.measure(qreg, creg)
 
    # Simulate
    simulator = AerSimulator()
    compiled = transpile(circuit, simulator)
    result = simulator.run(compiled, shots=shots).result()
    counts = result.get_counts()

    # Target success probability
    for item in marked_items:
        target = format(item, f'0{n_qubits}b')
        prob = counts.get(target, 0) / shots
        print(f"Success probability for |{target}>: {prob*100:.2f}%")
    return circuit, counts

